In [24]:
import pandas as pd
import numpy as np
import pickle
import csv

In [15]:
# Got an error when trying to unpickle the dict pickle below made on AWS.
# Solution source: https://github.com/pandas-dev/pandas/issues/16474#issuecomment-305566022 

import sys
# 1
import pandas.core.indexes 
sys.modules['pandas.indexes'] = pandas.core.indexes

In [2]:
pickles_path = "../../../pickles/"

In [6]:
cypher_path = "../../../cypher_scripts/"

### 1. Make edge files for graph viz

#### LINK_TO edges

In [7]:
inter_community_link_edges = pd.read_csv(cypher_path + "inter_community_link_edges.out")
print(len(inter_community_link_edges))
inter_community_link_edges.head()

5467


,from_lc_id,to_lc_id,link_edges_count,link_traffic
0,8,1,21717,1067594
1,3,1,62506,3543666
2,7,1,55577,3014753
3,7,2,51982,3602338
4,10,2,28916,2237781


In [108]:
# write to csv
inter_community_link_edges[["from_lc_id", "to_lc_id", "link_traffic"]].to_csv("community_link_edges.csv", 
                                                                                  index=False)

In [109]:
!head community_link_edges.csv

from_lc_id,to_lc_id,link_traffic
8,1,1067594
3,1,3543666
7,1,3014753
7,2,3602338
10,2,2237781
4,2,7302780
5,4,9931781
7,4,5441323
1,4,1557671


#### SEARCH_FOR edges

In [110]:
inter_community_search_edges = pd.read_csv(cypher_path + "inter_community_search_edges.out")
print(len(inter_community_search_edges))
inter_community_search_edges.head()

976


,from_lc_id,to_lc_id,search_edges_count,search_traffic
0,7,3,3596,86630
1,22,4,144,2745
2,2,4,1044,32692
3,14,7,10,190
4,10,7,701,21548


In [111]:
# write to csv
inter_community_search_edges[["from_lc_id", "to_lc_id", "search_traffic"]].to_csv("community_search_edges.csv", 
                                                                                  index=False)

In [112]:
!head community_search_edges.csv

from_lc_id,to_lc_id,search_traffic
7,3,86630
22,4,2745
2,4,32692
14,7,190
10,7,21548
4,7,141982
15,24,592
25,11,445
15,10,1868


### 2. Make a node file for graph viz

#### 2.1 Get community node ids and stats

In [47]:
# unpickle
with open(pickles_path + "en_1218_louvain_communities.pkl", 'rb') as picklefile: 
    louvain_communities = pickle.load(picklefile)

louvain_communities.head()

,articles_count,external_search_traffic,link_in_edges_count,link_in_traffic,louvain_community,search_in_edges_count,search_in_traffic,total_visits,link_edges_count,link_traffic,...,avg_external_search_traffic,avg_overall_link_in_degree,avg_community_link_in_degree,avg_link_in_traffic_per_edge,avg_link_traffic_per_edge,avg_visits_per_article,link_network_density,link_network_density_delta,network_density,network_density_delta
0,507642,535227964,3385462,329625267,3,102124,4188154,1124655652,2740230,279916754,...,1054.341374,6.668995,5.397958,97.364929,102.150825,2215.450361,0.000011,4.397968,0.000011,4.563288
1,302120,288453340,2037310,140416238,7,41383,1257742,592415339,1472052,100658886,...,954.764133,6.743380,4.872408,68.922372,68.379980,1960.861045,0.000016,3.872424,0.000016,3.964435
2,267541,626988532,2540296,461475849,4,98857,3938984,1350876078,2182385,423779216,...,2343.523168,9.494978,8.157198,181.662235,194.181694,5049.230129,0.000030,7.157229,0.000032,7.467672
3,233396,150495954,1209989,84128861,10,25894,932688,322074785,839295,54133551,...,644.809483,5.184275,3.596013,69.528616,64.498837,1379.949892,0.000015,2.596028,0.000016,2.671767
4,223864,219874886,1327054,151652674,5,32332,930331,468611692,1141537,132364916,...,982.180636,5.927947,5.099243,114.277696,115.953242,2093.287407,0.000023,4.099266,0.000023,4.206005


In [286]:
louvain_communities.columns

Index(['articles_count', 'external_search_traffic', 'link_in_edges_count',
       'link_in_traffic', 'louvain_community', 'search_in_edges_count',
       'search_in_traffic', 'total_visits', 'link_edges_count', 'link_traffic',
       'search_edges_count', 'search_traffic', 'avg_external_search_traffic',
       'avg_overall_link_in_degree', 'avg_community_link_in_degree',
       'avg_link_in_traffic_per_edge', 'avg_link_traffic_per_edge',
       'avg_visits_per_article', 'link_network_density',
       'link_network_density_delta', 'network_density',
       'network_density_delta'],
      dtype='object')

**Notes:**  

- `link_traffic` and `search_traffic` are internal traffic values within the specified community, while `link_in_traffic` and `search_in_traffic` are the sums of those values aggregated at nodes, and are not limited to the specified community (they include the traffic from other communities)

In [297]:
nodes_with_stats = louvain_communities[["louvain_community", "articles_count", "external_search_traffic",
                                        "link_traffic", "search_traffic",
                                        "link_in_traffic", "search_in_traffic",
                                        "total_visits", "network_density_delta"]].copy()

In [298]:
nodes_with_stats.head()

,louvain_community,articles_count,external_search_traffic,link_traffic,search_traffic,link_in_traffic,search_in_traffic,total_visits,network_density_delta
0,3,507642,535227964,279916754,3506684.0,329625267,4188154,1124655652,4.563288
1,7,302120,288453340,100658886,769867.0,140416238,1257742,592415339,3.964435
2,4,267541,626988532,423779216,3428064.0,461475849,3938984,1350876078,7.467672
3,10,233396,150495954,54133551,581906.0,84128861,932688,322074785,2.671767
4,5,223864,219874886,132364916,663439.0,151652674,930331,468611692,4.206005


#### 2.2 Add community topic info on nodes

In [19]:
# unpickle
with open(pickles_path + "en_1218_louvain_community_topics_dict.pkl", 'rb') as picklefile: 
    community_topics_dict = pickle.load(picklefile)

# community_topics_dict[22]

In [301]:
# testing
# community_topics_dict[22]["topic_ner"]

In [53]:
# testing
" ".join(community_topics_dict[22]["topic_words"][:5])

'grand prix championship seri engin'

In [302]:
# Filters out uninformative words
def filter_words(words):
    words_to_filter_out = ['list', 'disambigu', 'th']
    filtered_words = []
    for word in words:
        if not (word in words_to_filter_out):
            filtered_words.append(word)
    return filtered_words

In [303]:
# Add on the top 10 lemmatized topic words as a snapshot of the article community topic
nodes_with_stats["topic_snapshot"] = \
    [" ".join(filter_words(community_topics_dict[comm]["topic_words"])[:10]) 
     for comm in nodes_with_stats['louvain_community']]

In [304]:
# Add on the top 5 NERs
# SpaCy reference for NER types: https://spacy.io/api/annotation#named-entities
nodes_with_stats["topic_top_ner"] = \
    [" ".join(community_topics_dict[comm]["topic_ner"][:5]) for comm in nodes_with_stats['louvain_community']]

In [305]:
nodes_with_stats.head()

,louvain_community,articles_count,external_search_traffic,link_traffic,search_traffic,link_in_traffic,search_in_traffic,total_visits,network_density_delta,topic_snapshot,topic_top_ner
0,3,507642,535227964,279916754,3506684.0,329625267,4188154,1124655652,4.563288,unit state footbal elect nation constitu parti...,PERSON ORG GPE DATE NORP
1,7,302120,288453340,100658886,769867.0,140416238,1257742,592415339,3.964435,church al john novel saint languag histori st ...,PERSON ORG GPE NORP DATE
2,4,267541,626988532,423779216,3428064.0,461475849,3938984,1350876078,7.467672,film seri tv actor comic season award episod s...,PERSON ORG DATE GPE CARDINAL
3,10,233396,150495954,54133551,581906.0,84128861,932688,322074785,2.671767,state new counti school unit california york s...,PERSON GPE ORG DATE LOC
4,5,223864,219874886,132364916,663439.0,151652674,930331,468611692,4.206005,album song band discographi love music live mu...,PERSON ORG GPE DATE CARDINAL


#### 2.3 Add on node coordinates

The article communities network is much smaller than the articles network and can be rendered whole in a browser using vis.js, but it takes a long time for javascript to calculate the network's layout, and with each change while inspecting the visualization in the browser, the visualization will freeze up while javascript recalculates the node and edge positions for the network.  
To keep things fast and simple, we can avoid these rendering times and recalculations by rendering our graph once in vis.js (which can take a while), exporting the resulting node coordinates, and turning off physics in vis.js.  


The `node_locations_raw.txt` file contains the node coordinates exported from a graph rendered in vis.js.  
It was created by:
- running the vis.js rendering of the network in browser, where the vis network is stored in variable `network`  
- then running `network.stabilize();` in js console  
- then running `network.getPositions();` in js console  
- copying and pasting the js console output into a text editor  

The code below reads in the exported node coordinates and attaches them to the nodes dataset.

In [176]:
import re
from ast import literal_eval

In [227]:
# Read in the raw node locations file
with open ("node_locations_raw.txt", "r") as myfile:
    node_locations_raw=myfile.readlines()

In [229]:
# Node coordinates are ordered by community node id 
node_coords_ordered_list = []

# regex to put the dict keys inside quotes
to_replace = r'((?<={)|(?<=(, )))(\w+)(?=:)'
replace_with = r'"\3"'

i = 0
for line in node_locations_raw:
    coord = line.strip()
    coord = coord[(coord.find(":") + 1):]
    coord = re.sub(to_replace, replace_with, coord).strip()
    node_coords_ordered_list.append(literal_eval(coord))

    if i < 5:
        print(line.strip())
        print(coord)
    i += 1

0: {x: -205, y: 926}
{"x": -205, "y": 926}
1: {x: -264, y: 503}
{"x": -264, "y": 503}
2: {x: -611, y: 470}
{"x": -611, "y": 470}
3: {x: 729, y: -94}
{"x": 729, "y": -94}
4: {x: -804, y: -452}
{"x": -804, "y": -452}


In [306]:
nodes_with_stats["x"] = \
    [node_coords_ordered_list[comm]["x"] for comm in nodes_with_stats['louvain_community']]
nodes_with_stats["y"] = \
    [node_coords_ordered_list[comm]["y"] for comm in nodes_with_stats['louvain_community']]

In [307]:
nodes_with_stats.head()

,louvain_community,articles_count,external_search_traffic,link_traffic,search_traffic,link_in_traffic,search_in_traffic,total_visits,network_density_delta,topic_snapshot,topic_top_ner,x,y
0,3,507642,535227964,279916754,3506684.0,329625267,4188154,1124655652,4.563288,unit state footbal elect nation constitu parti...,PERSON ORG GPE DATE NORP,729,-94
1,7,302120,288453340,100658886,769867.0,140416238,1257742,592415339,3.964435,church al john novel saint languag histori st ...,PERSON ORG GPE NORP DATE,-132,-464
2,4,267541,626988532,423779216,3428064.0,461475849,3938984,1350876078,7.467672,film seri tv actor comic season award episod s...,PERSON ORG DATE GPE CARDINAL,-804,-452
3,10,233396,150495954,54133551,581906.0,84128861,932688,322074785,2.671767,state new counti school unit california york s...,PERSON GPE ORG DATE LOC,-628,511
4,5,223864,219874886,132364916,663439.0,151652674,930331,468611692,4.206005,album song band discographi love music live mu...,PERSON ORG GPE DATE CARDINAL,-1031,204


#### 2.4 Other  

##### Calculations for node shading based on network density delta

In [241]:
delta_max = nodes_with_stats.network_density_delta.max()
delta_min = nodes_with_stats.network_density_delta.min()
print("Max:", delta_max)
print("Min:", delta_min)

Max: 9.068592057761734
Min: -0.07692307692307684


In [240]:
delta_range = delta_max - delta_min
delta_range

9.14551513468481

In [262]:
round((10-(4))/10 * 230)

138

#### 2.5 Export the nodes dataset to csv

In [308]:
nodes_with_stats.to_csv("community_nodes.csv", index=False)

In [309]:
! head community_nodes.csv

louvain_community,articles_count,external_search_traffic,link_traffic,search_traffic,link_in_traffic,search_in_traffic,total_visits,network_density_delta,topic_snapshot,topic_top_ner,x,y
3,507642,535227964,279916754,3506684.0,329625267,4188154,1124655652,4.563287835300931,unit state footbal elect nation constitu parti battl john film,PERSON ORG GPE DATE NORP,729,-94
7,302120,288453340,100658886,769867.0,140416238,1257742,592415339,3.9644345440041837,church al john novel saint languag histori st film battl,PERSON ORG GPE NORP DATE,-132,-464
4,267541,626988532,423779216,3428064.0,461475849,3938984,1350876078,7.467672123794573,film seri tv actor comic season award episod school john,PERSON ORG DATE GPE CARDINAL,-804,-452
10,233396,150495954,54133551,581906.0,84128861,932688,322074785,2.6717667473596265,state new counti school unit california york station park district,PERSON GPE ORG DATE LOC,-628,511
5,223864,219874886,132364916,663439.0,151652674,930331,468611692,4.206005458695721,album 